# Business Challenge
- Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits
- provide a company name and their primary website

In [1]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display 
from openai import OpenAI

In [2]:
#initializing constants
llama_api = "http://localhost:11434/api/chat"
Headers = {"Content-Type": "application/json"}
Model = "llama3.1"

In [4]:
# Utility Class - Expand on website class from ollam website summary project
class Website:

    def __init__(self, url):
        self.url = url

        response = requests.get(url)
        #return content from url 
        self.body = response.content
        soup = BeautifulSoup(response.content, 'html.parser')

        self.title = soup.title.string if soup.title else "No Title"

        if soup.body:
            # remove unneeded objects from scrape for summarization
            for irrelevant_objects in soup.body(["script", "style", "img", "input"]):
                irrelevant_objects.decompose()
            self.text = soup.body.get_text(seperator="\n", strip=True)
        else:
            self.text = ""
        links = [links.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link] #Build a new list with only the truthy (non-empty, non-None, non-zero) items from links

    def get_content(self):
        return f"Webpage Title:\n{self.title}\n Webpage Contents:\n{self.text}\n\n"